#Bibs e Drive

In [1]:
import os
import re
import msgpack
try:
    from google.colab import drive
except Exception as e:
    print(f'Ocorreu um problema ao sincronizar com seu Google Drive:\n-->{e.__class__.__name__}: {str(e)}')
else:
    drive.mount('/content/drive')

Mounted at /content/drive


#Funções

In [2]:
def openMsgPackFile(full_filepath : str,
                    encoding_type : str = None):
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded

def formatarToken(token : str) -> str:
    return re.sub(r'[^\w\sáéíóúâêîôûàèìòùãõç]', '', token, flags=re.UNICODE).lower()

def formataFrase(frase : list) -> str:
    return ' '.join(frase).replace(' .','.')

def coletarFrasesDesejadas(token_desejado: str,
                           lista_de_frases_tokenizadas : list,
                           qtd_frases : int = 0) -> list:

    lista_de_indices_das_frases_desejadas = []

    tamanho_lista_frases = len(lista_de_frases_tokenizadas)

    for i,frase in enumerate(lista_de_frases_tokenizadas):
        for token in frase:
            token = formatarToken(token)
            if token == token_desejado:
                lista_de_indices_das_frases_desejadas.append(i)
                # print('achei token',token)
    # print('Encontrei já as frases com token desejado (se tiver)')
    lista_de_frases_desejadas = []

    for i in lista_de_indices_das_frases_desejadas:
        # print('Frase que encontrei o token:',formataFrase(lista_de_frases_tokenizadas[i]))
        lista_de_frases_encontradas = []
        if qtd_frases > 0:
            qtd_frases_vizinha_atras = qtd_frases
            while qtd_frases_vizinha_atras > 0:
                if i - qtd_frases_vizinha_atras < tamanho_lista_frases and i-qtd_frases_vizinha_atras >= 0:
                    lista_de_frases_encontradas.append(formataFrase(lista_de_frases_tokenizadas[i-qtd_frases_vizinha_atras]))
                qtd_frases_vizinha_atras -= 1
            # print('Preenchi já as vizinhas de trás',lista_de_frases_encontradas)
        lista_de_frases_encontradas.append(formataFrase(lista_de_frases_tokenizadas[i]))
        # print('Botei frase do meio',lista_de_frases_encontradas)
        if qtd_frases > 0:

            qtd_frases_vizinha_frente = 1
            while qtd_frases_vizinha_frente <= qtd_frases:
                if i + qtd_frases_vizinha_frente < tamanho_lista_frases:
                    lista_de_frases_encontradas.append(formataFrase(lista_de_frases_tokenizadas[i+qtd_frases_vizinha_frente]))
                qtd_frases_vizinha_frente += 1
            lista_de_frases_desejadas.append(lista_de_frases_encontradas)
            # print('Preenchi as vizinhas da frente')
    return lista_de_frases_desejadas

#Execução

In [4]:
frases_tokenizadas = openMsgPackFile(full_filepath=r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Resultados_Tokenizacao/Colecoes/Historia/2010/Trabalho 3/pure_tokens.msgpack')

token_desejado = input("Digite um token: ").lower()

quantidade_de_frases_vizinhas = input('Digite a quantidade de frases ao redor: ')
while not quantidade_de_frases_vizinhas.isdigit():
    quantidade_de_frases_vizinhas = input('Digite a quantidade de frases ao redor (número inteiro): ')
quantidade_de_frases_vizinhas = int(quantidade_de_frases_vizinhas)


resultado = coletarFrasesDesejadas(token_desejado=token_desejado,
                                   qtd_frases=quantidade_de_frases_vizinhas,
                                   lista_de_frases_tokenizadas=frases_tokenizadas)

if resultado:

    with open('resultado.txt','w',encoding='utf-8') as f:
        txt = 'Coleção História\nAno: 2010\n\n\nTrabalho: 3\n\n'
        for i,r in enumerate(resultado):
            print(i+1,r)
            txt += str(i+1)+'° conjunto de frases:\n'
            for index, frase in enumerate(r):
                if index != len(r)//2:
                    txt += frase+'\n'
                else:
                    txt += '--> '+frase+'\n'
            txt+='\n'
            print('\n')
        f.write(txt)
else:
    print('Sem resultados.')


Digite um token: branco
Digite a quantidade de frases ao redor: 10
1 ['O : I.......', '180 Ú : C F.............................', '191 F...........................................................................................', '197 R B...................................................', '199 A A........................................................................................', '215 AQUI COMEÇA A VIDA : INTRODUÇÃO A Um dos raros momentos de lazer da jovem trabalhadora Helen era assistir a filmes.', 'Para tentar aliviar o pesado trabalho em casa de família, a garota participava das sessões de cinema que ocorriam no interior do Village Hotel.', 'Em uma das noites de exibição de filmes, crianças espiavam curiosas escondidas atrás de uma cortina.', 'Um quadro que parecia retratar a chegada de Cristóvão Colombo a América tinha destaque na parede e, abaixo dele, podia ser lida a seguinte frase escrita em uma pequena placa: “Motion Pictures: the Wonder of the age”.', 'No hotel-cinem

## Como fica a tokenização mais "pura":

In [ ]:
frases_tokenizadas = openMsgPackFile(full_filepath=r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Resultados_Tokenizacao/Colecoes/Historia/2010/Trabalho 3/pure_tokens.msgpack')
for frase in frases_tokenizadas[150:200]:
    print(frase)
    print('')

['4', '.']

['O', ':', 'I', '.......']

['180', 'Ú', ':', 'C', 'F', '.............................']

['191', 'F', '...........................................................................................']

['197', 'R', 'B', '...................................................']

['199', 'A', 'A', '........................................................................................']

['215', 'AQUI', 'COMEÇA', 'A', 'VIDA', ':', 'INTRODUÇÃO', 'A', 'Um', 'dos', 'raros', 'momentos', 'de', 'lazer', 'da', 'jovem', 'trabalhadora', 'Helen', 'era', 'assistir', 'a', 'filmes.']

['Para', 'tentar', 'aliviar', 'o', 'pesado', 'trabalho', 'em', 'casa', 'de', 'família,', 'a', 'garota', 'participava', 'das', 'sessões', 'de', 'cinema', 'que', 'ocorriam', 'no', 'interior', 'do', 'Village', 'Hotel.']

['Em', 'uma', 'das', 'noites', 'de', 'exibição', 'de', 'filmes,', 'crianças', 'espiavam', 'curiosas', 'escondidas', 'atrás', 'de', 'uma', 'cortina.']

['Um', 'quadro', 'que', 'parecia', 'retratar', 

# Discussão acerca do espaço de armazenamento consumido

## Verificando tamanho ocupado pelos .txt salvos nas coleções da pasta de Extração:

In [ ]:
import os

caminho_pasta_colecoes = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

tamanho = 0

for colecao in [pasta_colecao for pasta_colecao in os.listdir(caminho_pasta_colecoes) if '.' not in pasta_colecao]:
    print(colecao)
    for ano in [pasta_ano for pasta_ano in os.listdir(os.path.join(caminho_pasta_colecoes,colecao)) if pasta_ano.isdigit()]:
        for trabalho in [pasta_trabalho for pasta_trabalho in os.listdir(os.path.join(caminho_pasta_colecoes,colecao,ano)) if pasta_trabalho.startswith('Trabalho')]:
            for arquivo in [arq for arq in os.listdir(os.path.join(caminho_pasta_colecoes,colecao,ano,trabalho)) if arq in ['texto_principal.txt','nota_de_rodape.txt']]:
                tamanho += os.path.getsize(os.path.join(caminho_pasta_colecoes,colecao,ano,trabalho,arquivo))


print("Tamanho do arquivo:", tamanho, "bytes")
print("Tamanho do arquivo:", tamanho/(1024), "KB")
print("Tamanho do arquivo:", tamanho/(1024*1024), "MB")
print("Tamanho do arquivo:", tamanho/(1024*1024*1024), "GB")


Administracao
Arquitetura_e_Urbanismo
Ciencia_da_Informacao
Direito
Educacao
Enfermagem
Ciencia_dos_Alimentos
Assistencia_Farmaceutica
Biologia_Celular_e_do_Desenvolvimento
Biologia_de_Fungos_Algas_e_Plantas
Ciencia_e_Engenharia_de_Materiais
Administracao_Universitaria_Mestrado_Profissional
Biologia_Vegetal
Engenharia_Ambiental
Bioquimica
Agroecossistemas
Biotecnologia
Biotecnologia_e_Biociencias
Ciencias_Medicas
Ciencia_da_Computacao
Economia
Agroecossistemas_Mestrado_Profissional
Antropologia_Social
Contabilidade
Educacao_Fisica
Engenharia_de_Automacao_e_Sistemas
Educacao_Cientifica_e_Tecnologica
Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional
Desastres_Naturais_Mestrado_Profissional
Engenharia_e_Ciencias_Mecanicas
Design
Engenharia_Mecanica_Mestrado_Profissional
Engenharia_Textil
Aquicultura
Ensino_de_Biologia_Mestrado_Profissional
Engenharia_de_Alimentos
Ciencias_da_Reabilitacao
Design_e_Expressao_Grafica
Ensino_de_Biologia
Ensino_de_Historia_Mestrado_Profissional
Controle_d

### Conferindo análise

In [ ]:
t = '''Administracao
Arquitetura_e_Urbanismo
Ciencia_da_Informacao
Direito
Educacao
Enfermagem
Ciencia_dos_Alimentos
Assistencia_Farmaceutica
Biologia_Celular_e_do_Desenvolvimento
Biologia_de_Fungos_Algas_e_Plantas
Ciencia_e_Engenharia_de_Materiais
Administracao_Universitaria_Mestrado_Profissional
Biologia_Vegetal
Engenharia_Ambiental
Bioquimica
Agroecossistemas
Biotecnologia
Biotecnologia_e_Biociencias
Ciencias_Medicas
Ciencia_da_Computacao
Economia
Agroecossistemas_Mestrado_Profissional
Antropologia_Social
Contabilidade
Educacao_Fisica
Engenharia_de_Automacao_e_Sistemas
Educacao_Cientifica_e_Tecnologica
Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional
Desastres_Naturais_Mestrado_Profissional
Engenharia_e_Ciencias_Mecanicas
Design
Engenharia_Mecanica_Mestrado_Profissional
Engenharia_Textil
Aquicultura
Ensino_de_Biologia_Mestrado_Profissional
Engenharia_de_Alimentos
Ciencias_da_Reabilitacao
Design_e_Expressao_Grafica
Ensino_de_Biologia
Ensino_de_Historia_Mestrado_Profissional
Controle_de_Gestao_Mestrado_Profissional
Ensino_de_Fisica_Mestrado_Profissional
Engenharia_Ambiental_Mestrado_Profissional
Engenharia_de_Sistemas_Eletronicos
Engenharia_de_Transportes_e_Gestao_Territorial
Direito_Mestrado_Profissional
Farmacologia_Mestrado_Profissional
Fonoaudiologia
Gestao_do_Cuidado_em_Enfermagem
Geologia
Ecologia
Gestao_do_Cuidado_em_Enfermagem_Mestrado_Profissional
Ecossistemas_Agricolas_e_Naturais
Energia_e_Sustentabilidade
Engenharia_Civil
Engenharia_Eletrica
Engenharia_Mecanica
Farmacia
Letras_Mestrado_Profissional
Letras_Ingles_e_Literatura_Correspondente
Letras_Literatura_Brasileira
Linguistica
Filosofia
Geografia
Literatura
Engenharia_e_Gestao_do_Conhecimento
Farmacologia
Historia
Engenharia_Quimica
Matematica_e_Computacao_Cientifica
Fisica
Matematica_Mestrado_Profissional
Estudos_da_Traducao
Medicina_Veterinaria_Convencional_e_Integrativa
Matematica_Pura_e_Aplicada
Metrologia_Cientifica_e_Industrial
Nanociencia_Processos_e_Materiais_Avancados
Metodos_e_Gestao_em_Avaliacao_Mestrado_Profissional
Ingles_Estudos_Linguisticos_e_Literarios
Informatica_em_Saude_Mestrado_Profissional
Jornalismo
Engenharia_de_Producao
Nanotecnologia_Farmaceutica
Quimica
Neurociencias
Tecnologias_da_Informacao_e_Comunicacao
Teses_e_dissertacoes_nao_defendidas_na_UFSC
Urbanismo_Historia_e_Arquitetura_da_Cidade
Teses_e_dissertacoes_do_Centro_Tecnologico
Nutricao
Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas
Oceanografia
Programa_de_Pos_Graduacao_Multicentrico_em_Ciencias_Fisiologicas
Odontologia
Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional
Teses_e_dissertacoes_do_Centro_de_Ciencias_Agrarias
Teses_e_dissertacoes_do_Centro_de_Ciencias_da_Saude
Recursos_Geneticos_Vegetais
Teses_e_dissertacoes_do_Centro_de_Ciencias_Fisicas_e_Matematicas
Pericias_Criminais_Ambientais_Mestrado_Profissional
Saude_Coletiva
Teses_e_dissertacoes_do_Centro_de_Comunicacao_e_Expressao
Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
Psicologia
Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas
Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional
Saude_Publica
Teses_e_dissertacoes_do_Centro_Socio_Economico
Sociologia_e_Ciencia_Politica
Servico_Social
Relacoes_Internacionais
Sociologia_Politica'''

print(len(t.split('\n')))

112


## Verificando tamanho ocupado pelos .msgpack salvos nas coleções da pasta de Word Embeddings:

In [ ]:
import os

caminho_pasta_colecoes = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Resultados_Tokenizacao/Colecoes'

tamanho = 0

for colecao in [pasta_colecao for pasta_colecao in os.listdir(caminho_pasta_colecoes) if '.' not in pasta_colecao]:
    print(colecao)
    for ano in [pasta_ano for pasta_ano in os.listdir(os.path.join(caminho_pasta_colecoes,colecao)) if pasta_ano.isdigit()]:
        for trabalho in [pasta_trabalho for pasta_trabalho in os.listdir(os.path.join(caminho_pasta_colecoes,colecao,ano)) if pasta_trabalho.startswith('Trabalho')]:
            try:
                tamanho += os.path.getsize(os.path.join(caminho_pasta_colecoes,colecao,ano,trabalho,'pure_tokens.msgpack'))
            except Exception as e:
                print(f'! Problema: {e.__class__.__name__}: {str(e)}')


print("Tamanho do arquivo:", tamanho, "bytes")
print("Tamanho do arquivo:", tamanho/(1024), "KB")
print("Tamanho do arquivo:", tamanho/(1024*1024), "MB")
print("Tamanho do arquivo:", tamanho/(1024*1024*1024), "GB")


Historia
Administracao
Administracao_Universitaria_Mestrado_Profissional
Agroecossistemas
Agroecossistemas_Mestrado_Profissional
Antropologia_Social
Ciencias_Medicas
Engenharia_Ambiental
Ensino_de_Historia_Mestrado_Profissional
Letras_Mestrado_Profissional
Estudos_da_Traducao
Ciencias_da_Reabilitacao
Linguistica
Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional
Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
Contabilidade
Psicologia
Aquicultura
Engenharia_Ambiental_Mestrado_Profissional
Engenharia_Civil
Controle_de_Gestao_Mestrado_Profissional
Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional
Desastres_Naturais_Mestrado_Profissional
Literatura
Design
Arquitetura_e_Urbanismo
Design_e_Expressao_Grafica
Quimica
Direito
Engenharia_Eletrica
Farmacia
Assistencia_Farmaceutica
Biologia_Celular_e_do_Desenvolvimento
Matematica_Mestrado_Profissional
Matematica_Pura_e_Aplicada
Biologia_Vegetal
Matematica_e_Computacao_Cientifica

### Conferindo análise

In [ ]:
t = '''Historia
Administracao
Administracao_Universitaria_Mestrado_Profissional
Agroecossistemas
Agroecossistemas_Mestrado_Profissional
Antropologia_Social
Ciencias_Medicas
Engenharia_Ambiental
Ensino_de_Historia_Mestrado_Profissional
Letras_Mestrado_Profissional
Estudos_da_Traducao
Ciencias_da_Reabilitacao
Linguistica
Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional
Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
Contabilidade
Psicologia
Aquicultura
Engenharia_Ambiental_Mestrado_Profissional
Engenharia_Civil
Controle_de_Gestao_Mestrado_Profissional
Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional
Desastres_Naturais_Mestrado_Profissional
Literatura
Design
Arquitetura_e_Urbanismo
Design_e_Expressao_Grafica
Quimica
Direito
Engenharia_Eletrica
Farmacia
Assistencia_Farmaceutica
Biologia_Celular_e_do_Desenvolvimento
Matematica_Mestrado_Profissional
Matematica_Pura_e_Aplicada
Biologia_Vegetal
Matematica_e_Computacao_Cientifica
Biologia_de_Fungos_Algas_e_Plantas
Medicina_Veterinaria_Convencional_e_Integrativa
Metodos_e_Gestao_em_Avaliacao_Mestrado_Profissional
Metrologia_Cientifica_e_Industrial
Recursos_Geneticos_Vegetais
Bioquimica
Nanociencia_Processos_e_Materiais_Avancados
Nanotecnologia_Farmaceutica
Neurociencias
Biotecnologia
Relacoes_Internacionais
Nutricao
Biotecnologia_e_Biociencias
Saude_Coletiva
Ciencia_da_Computacao
Engenharia_Mecanica
Oceanografia
Odontologia
Farmacologia
Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional
Direito_Mestrado_Profissional
Saude_Publica
Ecologia
Servico_Social
Ciencia_da_Informacao
Pericias_Criminais_Ambientais_Mestrado_Profissional
Economia
Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas
Sociologia_Politica
Farmacologia_Mestrado_Profissional
Ciencia_dos_Alimentos
Filosofia
Programa_de_Pos_Graduacao_Multicentrico_em_Ciencias_Fisiologicas
Ecossistemas_Agricolas_e_Naturais
Educacao
Engenharia_Mecanica_Mestrado_Profissional
Engenharia_Quimica
Sociologia_e_Ciencia_Politica
Ciencia_e_Engenharia_de_Materiais
Tecnologias_da_Informacao_e_Comunicacao
Teses_e_dissertacoes_do_Centro_Socio_Economico
Teses_e_dissertacoes_do_Centro_Tecnologico
Teses_e_dissertacoes_do_Centro_de_Ciencias_Agrarias
Teses_e_dissertacoes_do_Centro_de_Ciencias_Fisicas_e_Matematicas
Teses_e_dissertacoes_do_Centro_de_Ciencias_da_Saude
Teses_e_dissertacoes_do_Centro_de_Comunicacao_e_Expressao
Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas
Teses_e_dissertacoes_nao_defendidas_na_UFSC
Urbanismo_Historia_e_Arquitetura_da_Cidade
Engenharia_Textil
Engenharia_de_Alimentos
Fisica
Engenharia_de_Automacao_e_Sistemas
Engenharia_de_Producao
Fonoaudiologia
Geografia
Educacao_Cientifica_e_Tecnologica
Geologia
Gestao_do_Cuidado_em_Enfermagem
Gestao_do_Cuidado_em_Enfermagem_Mestrado_Profissional
Educacao_Fisica
Engenharia_de_Sistemas_Eletronicos
Engenharia_de_Transportes_e_Gestao_Territorial
Engenharia_e_Ciencias_Mecanicas
Engenharia_e_Gestao_do_Conhecimento
Informatica_em_Saude_Mestrado_Profissional
Ingles_Estudos_Linguisticos_e_Literarios
Jornalismo
Energia_e_Sustentabilidade
Enfermagem
Letras_Ingles_e_Literatura_Correspondente
Letras_Literatura_Brasileira
Ensino_de_Biologia
Ensino_de_Biologia_Mestrado_Profissional
Ensino_de_Fisica_Mestrado_Profissional'''

print(len(t.split('\n')))

112
